In [2]:
import pandas as pd

In [3]:
proc = pd.read_csv("procedures_preprocess_threshold_nonfiltered.csv")

In [ ]:
diag = pd.read_csv("diagnosis_preprocess.csv")
#"data/diagnosis_preprocess_nonfiltered.csv"
drugs = pd.read_csv("drugs_preprosss.csv")

In [12]:
adm = pd.read_csv("/Users/cgarciay/Desktop/Laval_Master_Computer/research/MIMIC/ADMISSIONS.csv.gz")


# Adm

In [23]:
import polars as pl
#df = pl.read_parquet("df_filtered.parquet")
df =pl.read_parquet("df_non_filtered.parquet")
adm = df[[
 'ADMISSION_TYPE',
 'ADMISSION_LOCATION',
 'DISCHARGE_LOCATION',
 'INSURANCE',
 'LANGUAGE',
 'RELIGION',
 'MARITAL_STATUS',
 'ETHNICITY',
 'DIAGNOSIS',
 'HOSPITAL_EXPIRE_FLAG',

 'GENDER',
 'EXPIRE_FLAG',

  'DOB','SUBJECT_ID',
 'HADM_ID','ADMITTIME','DISCHTIME'

]].to_pandas()

adm["DISCHTIME"] = pd.to_datetime(adm["DISCHTIME"])
adm["ADMITTIME"] = pd.to_datetime(adm["ADMITTIME"])
adm["LOSRD"] = adm["DISCHTIME"] - adm["ADMITTIME"]

In [109]:
''''''adm = pd.read_csv("dataset/ADMISSIONS.csv.gz")
desired_ids = pd.read_csv("ids.csv")["Subjects_id"]
condition = adm['SUBJECT_ID'].isin(desired_ids)

# Apply the condition to filter the DataFrame
adm = adm[condition]
''''''

In [25]:
adm['LOSRD']  = [i.days for i in adm['LOSRD']]

In [27]:
adm['DOB'] = pd.to_datetime(adm['DOB'])
adm['ADMITTIME'] = pd.to_datetime(adm['ADMITTIME'])
adm['ADMITTIME'] = [timestamp.to_pydatetime() for timestamp in adm['ADMITTIME']]
adm['DOB'] = [timestamp.to_pydatetime() for timestamp in adm['DOB']]
adm['ADMITTIME'] = [timestamp.date() for timestamp in adm['ADMITTIME']]
adm['DOB'] = [timestamp.date() for timestamp in adm['DOB']]
adm["age"] = (adm['ADMITTIME'].to_numpy() - adm['DOB'].to_numpy())
adm["year_age"] = [i.days/365 for i in adm["age"]]

Replace does person that have more than 90 years.

In [28]:
def replace_age(age):
    if age > 100:
        return 90
    else:
        return age

# Apply the custom function to the 'Age' column
adm['year_age'] = adm['year_age'].apply(replace_age)


In [29]:
# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = adm.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_adm = grouped.reset_index()
grouped_adm

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,EXPIRE_FLAG,DOB,ADMITTIME,DISCHTIME,LOSRD,age,year_age
0,10,184167,[NEWBORN],[PHYS REFERRAL/NORMAL DELI],[SHORT TERM HOSPITAL],[Medicaid],[None],[UNOBTAINABLE],[None],[BLACK/AFRICAN AMERICAN],[NEWBORN],[0],[F],[0],[2103-06-28],[2103-06-28],[2103-07-06 12:10:00],[8],[0:00:00],[0.0]
1,100,153952,[ELECTIVE],[PHYS REFERRAL/NORMAL DELI],[HOME],[Private],[None],[CATHOLIC],[WIDOWED],[UNKNOWN/NOT SPECIFIED],[AORTIC INSUFFICIENCY\AORTIC VALVE REPLACEMENT...,[0],[F],[0],[2085-08-31],[2157-08-10],[2157-08-18 19:54:00],[8],"[26276 days, 0:00:00]",[71.98904109589041]
2,1000,143040,[URGENT],[TRANSFER FROM HOSP/EXTRAM],[DEAD/EXPIRED],[Private],[None],[UNOBTAINABLE],[MARRIED],[UNKNOWN/NOT SPECIFIED],[HEART FAILURE],[1],[M],[1],[2074-05-06],[2144-01-19],[2144-02-25 06:05:00],[36],"[25459 days, 0:00:00]",[69.75068493150685]
3,10000,187813,[EMERGENCY],[TRANSFER FROM HOSP/EXTRAM],[SHORT TERM HOSPITAL],[Self Pay],[SPAN],[CATHOLIC],[None],[HISPANIC OR LATINO],[END STAGE LIVER DISEASE;END STAGE KIDNEY DISE...,[0],[M],[0],[2136-12-12],[2186-08-10],[2186-08-31 17:00:00],[20],"[18138 days, 0:00:00]",[49.69315068493151]
4,10001,118420,[NEWBORN],[PHYS REFERRAL/NORMAL DELI],[HOME],[Government],[None],[BUDDHIST],[None],[ASIAN],[NEWBORN],[0],[M],[0],[2107-03-18],[2107-03-18],[2107-03-20 11:59:00],[1],[0:00:00],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,9999,117856,[EMERGENCY],[TRANSFER FROM HOSP/EXTRAM],[DEAD/EXPIRED],[Medicaid],[None],[CATHOLIC],[SINGLE],[UNKNOWN/NOT SPECIFIED],[LIVER FAILURE],[1],[M],[1],[2091-02-27],[2134-12-26],[2134-12-28 04:45:00],[1],"[16007 days, 0:00:00]",[43.85479452054795]
58972,99991,151118,[ELECTIVE],[PHYS REFERRAL/NORMAL DELI],[HOME],[Private],[ENGL],[NOT SPECIFIED],[MARRIED],[WHITE],[DIVERTICULITIS/SDA],[0],[M],[0],[2137-04-07],[2184-12-24],[2185-01-05 12:15:00],[12],"[17428 days, 0:00:00]",[47.74794520547945]
58973,99992,197084,[EMERGENCY],[CLINIC REFERRAL/PREMATURE],[SNF],[Medicare],[ENGL],[CATHOLIC],[WIDOWED],[WHITE],[RETROPERITONEAL HEMORRHAGE],[0],[F],[0],[2078-10-17],[2144-07-25],[2144-07-28 17:56:00],[2],"[24022 days, 0:00:00]",[65.81369863013698]
58974,99995,137810,[ELECTIVE],[PHYS REFERRAL/NORMAL DELI],[HOME],[Medicare],[ENGL],[NOT SPECIFIED],[MARRIED],[WHITE],[ABDOMINAL AORTIC ANEURYSM/SDA],[0],[F],[1],[2058-05-29],[2147-02-08],[2147-02-11 13:15:00],[3],"[32396 days, 0:00:00]",[88.75616438356164]


In [30]:
grouped_adm.HADM_ID.nunique()

58976

In [31]:
grouped_adm.SUBJECT_ID.nunique()

46520

# Procedure

In [32]:
proc.rename(columns={ "'CCS CATEGORY DESCRIPTION'":'CCS CATEGORY DESCRIPTION_proc',
        'CCS CODES':'CCS CODES_proc','cat_threshold .95 most frequent': 'cat_threshold .95 most frequent_proc',"ICD9_CODE":"ICD9_CODE_procedures"},inplace = True)

In [33]:
proc = proc[['SUBJECT_ID', 'HADM_ID', "ICD9_CODE_procedures",  'CCS CATEGORY DESCRIPTION_proc',
        'CCS CODES_proc',
       'cat_threshold .95 most frequent_proc']]

In [34]:

# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = proc.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_proc = grouped.reset_index()
grouped_proc

,SUBJECT_ID,HADM_ID,ICD9_CODE_procedures,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc
0,2,163353,[9955],['Vaccinations '],['228'],[9955]
1,3,145834,"[3893, 9672, 9604, 966, 9962, 8964]","['Ot vasc cath ', 'Mech ventil ', 'Me...","['54', '216', '216', '223', '225', '204']","[3893, 9672, 9604, 966, 9962, 8964]"
2,4,185777,"[3893, 8872, 3323]","['Ot vasc cath ', 'Echocardiogr ', 'Br...","['54', '193', '37']","[3893, 8872, 3323]"
3,5,178980,[9955],['Vaccinations '],['228'],[9955]
4,6,107064,"[5569, 9904, 3957, 3806, 91]","['Kidney trans ', 'Blood transf ', 'Ot...","['105', '222', '61', '61', '231']","[5569, 9904, 3957, -1, -1]"
...,...,...,...,...,...,...
52238,99985,176670,"[3893, 9672, 9604, 966, 14]","['Ot vasc cath ', 'Mech ventil ', 'Me...","['54', '216', '216', '223', '231']","[3893, 9672, 9604, 966, 14]"
52239,99991,151118,"[5491, 544, 5459, 5412, 4823, 5425, 1736]","['Abd paracent ', 'OR GI Rx prs ', 'Ex...","['88', '99', '90', '99', '77', '97', '78']","[5491, 544, 5459, 5412, 4823, 5425, -1]"
52240,99992,197084,"[8847, 3979]","['Ot arterio ', 'Ot vessels ']","['191', '61']","[8847, 3979]"
52241,99995,137810,"[3971, 8339, 1771]","['Aort resect ', 'Rx prs musc ', 'Ot...","['52', '160', '61']","[3971, 8339, -1]"


In [21]:
grouped_proc.HADM_ID.nunique()

17393

In [35]:
grouped_proc.SUBJECT_ID.nunique()

42214

# Diagnosis

In [23]:
diag

,Unnamed: 0,SUBJECT_ID,HADM_ID,ICD9_CODE,CCS CODES,LEVE3 CODES,3_digitos,icd9_category,cat_threshold .75,'CCS CATEGORY DESCRIPTION'
0,0,23,152223,41401,101,414,414,Diseases of the Circulatory System,41401,'Coron athero'
1,1,23,152223,4111,101,411,411,Diseases of the Circulatory System,4111,'Coron athero'
2,2,23,152223,4241,96,424,424,Diseases of the Circulatory System,4241,'Hrt valve dx'
3,3,23,152223,V4582,101,V45,1000,External causes of injury and supplemental cla...,V4582,'Coron athero'
4,4,23,152223,2724,53,272,272,"Endocrine, Nutritional, and Metabolic Diseases...",2724,'Hyperlipidem'
...,...,...,...,...,...,...,...,...,...,...
260277,260277,98813,190264,311,657,311,311,Mental Disorders,311,'Mood disorders'
260278,260278,98813,190264,30390,660,303,303,Mental Disorders,30390,'Alcohol-related disorders'
260279,260279,98813,190264,2948,653,294,294,Mental Disorders,2948,'Delirium/dementia/amnestic/other cognitiv'
260280,260280,98813,190264,V1588,244,V15,1000,External causes of injury and supplemental cla...,V1588,'Other injury'


In [24]:
diag.rename(columns={ "ICD9_CODE":"ICD9_CODE_diagnosis"},inplace = True)

In [25]:
diag = diag[[ 'SUBJECT_ID', 'HADM_ID',  'CCS CODES',
       'LEVE3 CODES',  'icd9_category', 'cat_threshold .75',"'CCS CATEGORY DESCRIPTION'" ,"ICD9_CODE_diagnosis"]]

In [26]:
diag.rename(columns={ 'CCS CODES':'CCS CODES_diag',
        'cat_threshold .75':'cat_threshold .75_diag',"'CCS CATEGORY DESCRIPTION'": "CCS CATEGORY DESCRIPTION_diag"},inplace = True)

/Users/cgarciay/opt/anaconda3/envs/luigi/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [27]:
diag

,SUBJECT_ID,HADM_ID,CCS CODES_diag,LEVE3 CODES,icd9_category,cat_threshold .75_diag,CCS CATEGORY DESCRIPTION_diag,ICD9_CODE_diagnosis
0,23,152223,101,414,Diseases of the Circulatory System,41401,'Coron athero',41401
1,23,152223,101,411,Diseases of the Circulatory System,4111,'Coron athero',4111
2,23,152223,96,424,Diseases of the Circulatory System,4241,'Hrt valve dx',4241
3,23,152223,101,V45,External causes of injury and supplemental cla...,V4582,'Coron athero',V4582
4,23,152223,53,272,"Endocrine, Nutritional, and Metabolic Diseases...",2724,'Hyperlipidem',2724
...,...,...,...,...,...,...,...,...
260277,98813,190264,657,311,Mental Disorders,311,'Mood disorders',311
260278,98813,190264,660,303,Mental Disorders,30390,'Alcohol-related disorders',30390
260279,98813,190264,653,294,Mental Disorders,2948,'Delirium/dementia/amnestic/other cognitiv',2948
260280,98813,190264,244,V15,External causes of injury and supplemental cla...,V1588,'Other injury',V1588


In [28]:

# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = diag.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_diag = grouped.reset_index()
grouped_diag

,SUBJECT_ID,HADM_ID,CCS CODES_diag,LEVE3 CODES,icd9_category,cat_threshold .75_diag,CCS CATEGORY DESCRIPTION_diag,ICD9_CODE_diagnosis
0,17,161087,"[97, 130, 249, 117, 657, 205, 204, 53]","[423, 511, 785, 458, 311, 722, 719, 272]","[Diseases of the Circulatory System, Diseases ...","[Otro, 5119, 78551, 4589, 311, Otro, 71946, 2724]","['Carditis', 'Pleurisy', 'Shock', 'Ot circul d...","[4239, 5119, 78551, 4589, 311, 7220, 71946, 2724]"
1,17,194023,"[213, 238, 117, 53]","[745, 458, V12, 272]","[Congenital Anomalies, Diseases of the Circula...","[7455, 45829, V1259, 2724]","['Cardiac anom', 'Complic proc', 'Ot circul dx...","[7455, 45829, V1259, 2724]"
2,21,109451,"[100, 249, 153, 157, 99, 108, 121, 129, 106, 1...","[410, 785, 578, 584, 403, 428, 459, 507, 427, ...","[Diseases of the Circulatory System, Symptoms,...","[41071, 78551, 5781, 5849, 40391, 4280, Otro, ...","['Acute MI', 'Shock', 'GI hemorrhag', 'Ac renl...","[41071, 78551, 5781, 5849, 40391, 4280, 4592, ..."
3,21,111970,"[2, 249, 99, 106, 199, 130, 197, 238, 135, 151...","[038, 785, 403, 427, 707, 511, 682, 998, 008, ...","[Infectious and Parasitic Diseases, Symptoms, ...","[Otro, 78552, 40391, 42731, 70709, 5119, Otro,...","['Septicemia', 'Shock', 'Htn complicn', 'Dysrh...","[0388, 78552, 40391, 42731, 70709, 5119, 6823,..."
4,23,124321,"[47, 95, 83, 96, 98, 53, 53, 101, 259, 663]","[225, 348, 780, 424, 401, 272, 272, V45, V45, ...","[Neoplasms, Diseases of the Nervous System and...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","['Ot bnign neo', 'Oth nerv dx', 'Epilepsy/cnv'...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4..."
...,...,...,...,...,...,...,...,...
19944,99923,164914,"[238, 118, 55, 151, 138, 138, 157, 213, 2617, ...","[458, 453, 276, 572, 456, 456, 584, 745, E93, ...","[Diseases of the Circulatory System, Diseases ...","[45829, Otro, 2761, 5723, Otro, Otro, 5849, 74...","['Complic proc', 'Phlebitis', 'Fluid/elc dx', ...","[45829, 4532, 2761, 5723, 4561, 45621, 5849, 7..."
19945,99923,192053,"[660, 158, 151, 99, 238, 145, 660, 118, 257, 6...","[571, 585, 572, 403, 997, 560, 303, V12, V58, ...","[Diseases of the Digestive System, Diseases of...","[5712, 5856, 5724, 40391, 9974, 5601, 30393, V...","['Alcohol-related disorders', 'Chr kidney dise...","[5712, 5856, 5724, 40391, 9974, 5601, 30393, V..."
19946,99982,112748,"[108, 108, 157, 97, 55, 106, 155, 138, 96, 257...","[428, 428, 584, 425, 276, 427, 787, 530, V42, ...","[Diseases of the Circulatory System, Diseases ...","[4280, 42823, 5849, 4254, Otro, 42731, Otro, 5...","['chf;nonhp', 'chf;nonhp', 'Ac renl fail', 'Ca...","[4280, 42823, 5849, 4254, 2763, 42731, 78729, ..."
19947,99982,151454,"[108, 97, 62, 106, 96, 138, 108, 257, 238]","[428, 425, 287, 427, 397, 530, 428, V58, 458]","[Diseases of the Circulatory System, Diseases ...","[42823, 4254, 2875, 42731, 3970, 5303, 4280, V...","['chf;nonhp', 'Carditis', 'Coag/hemr dx', 'Dys...","[42823, 4254, 2875, 42731, 3970, 5303, 4280, V..."


# Drugs

In [29]:
drugs.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'DRUG', 'ATC4',
       'ATC3'],
      dtype='object')

In [30]:
drugs.shape

(2078616, 7)

In [31]:
drugs = drugs[[ 'SUBJECT_ID', 'HADM_ID' , 'DRUG', 'ATC4', 'ATC3']]

In [32]:

# Group by 'SUBJECT_ID' and 'HADM_ID' and aggregate all columns into lists
grouped = drugs.groupby(['SUBJECT_ID', 'HADM_ID']).agg(lambda x: x.tolist())

# Reset the index to make 'SUBJECT_ID' and 'HADM_ID' regular columns
grouped_drugs = grouped.reset_index()
grouped_drugs

,SUBJECT_ID,HADM_ID,DRUG,ATC4,ATC3
0,2,163353,[Ampicillin Sodium],[J01CA],[J01C]
1,4,185777,"[Insulin, Lantus, Lantus, Insulin, Lantus, Ins...","[A10AE, A10AE, A10AE, A10AE, A10AE, A10AE, A10...","[A10A, A10A, A10A, A10A, A10A, A10A, A10A, A10..."
2,6,107064,"[Docusate Sodium, NS, NS, NS, NS, Anti-Thymocy...","[A06AA, A12CA, A12CA, A12CA, A12CA, L04AA, L04...","[A06A, A12C, A12C, A12C, A12C, L04A, L04A, L04..."
3,8,159514,"[NEO*IV*Ampicillin Sodium, NEO*IV*Gentamicin]","[J01CA, D06AX]","[J01C, D06A]"
4,9,150750,"[Potassium Chloride, D5W, D5W, D5W, Metoprolol...","[A12BA, B05CX, B05CX, B05CX, C07AB, A12AA, A12...","[A12B, B05C, B05C, B05C, C07A, A12A, A12C, J01..."
...,...,...,...,...,...
49924,99985,176670,"[Heparin, 5% Dextrose, 5% Dextrose, 0.9% Sodiu...","[B01AB, B05CX, B05CX, A12CA, A12CA, A12CA, A12...","[B01A, B05C, B05C, A12C, A12C, A12C, A12C, A12..."
49925,99991,151118,"[Heparin, Heparin, Heparin, 5% Dextrose, 0.9% ...","[B01AB, B01AB, B01AB, B05CX, A12CA, A12CA, C03...","[B01A, B01A, B01A, B05C, A12C, A12C, C03C, C03..."
49926,99992,197084,"[Acetaminophen, traZODONE, Dextrose 50%, Loraz...","[N02BE, N06AX, B05CX, N05BA, B03BB, A02AC, C09...","[N02B, N06A, B05C, N05B, B03B, A02A, C09C, C09..."
49927,99995,137810,"[Heparin, Heparin, Metoprolol Tartrate, Acetam...","[B01AB, B01AB, C07AB, N02BE, C02DB, A07AA, A07...","[B01A, B01A, C07A, N02B, C02D, A07A, A07A, A01..."


In [33]:
adm.shape

(19993, 23)

In [34]:
proc.shape

(81755, 6)

In [36]:
adm['SUBJECT_ID'] = adm['SUBJECT_ID'].astype(int)
adm['HADM_ID'] = adm['HADM_ID'].astype(int)

In [37]:
merge_proc = pd.merge(adm, grouped_proc, on=['SUBJECT_ID', 'HADM_ID'], how='left')
merge_proc

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,...,HADM_ID,ADMITTIME,DISCHTIME,LOSRD,age,year_age,ICD9_CODE_procedures,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc
0,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,0,...,165315,2196-04-09,2196-04-10 15:54:00,1,"23714 days, 0:00:00",64.969863,"[9671, 9604, 9634]","['Mech ventil ', 'Mech ventil ', 'nO...","['216', '216', '93']","[9671, 9604, 9634]"
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,152223,2153-09-03,2153-09-08 19:10:00,5,"25980 days, 0:00:00",71.178082,"[3613, 3615, 3961, 8872, 9904, 9907, 9905]","['CABG ', 'CABG ', 'Ex...","['44', '44', '50', '193', '222', '222', '222']","[3613, 3615, 3961, 8872, 9904, 9907, 9905]"
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,...,124321,2157-10-18,2157-10-25 14:00:00,6,"27486 days, 0:00:00",75.304110,[151],['Inc/exc CNS '],['1'],[151]
3,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,...,161859,2139-06-06,2139-06-09 12:48:00,2,"14250 days, 0:00:00",39.041096,"[3601, 3723, 8856, 9920, 8852, 3607]","['PTCA ', 'Cardiac cath ', 'Ca...","['45', '47', '47', '231', '47', '63']","[3601, 3723, 8856, 9920, 8852, 3607]"
4,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,0,...,129635,2160-11-02,2160-11-05 14:55:00,3,"21531 days, 0:00:00",58.989041,"[3601, 8856, 8856, 8853, 9920, 9920, 3722, 372...","['PTCA ', 'Cardiac cath ', 'Ca...","['45', '47', '47', '47', '231', '231', '47', '...","[3601, 8856, 8856, 8853, 9920, 9920, 3722, 372..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,0,...,191113,2131-03-30,2131-04-02 15:02:00,2,"7085 days, 0:00:00",19.410959,"[9671, 2751, 881, 8659]","['Mech ventil ', 'nOR nose mou ', 'Ot...","['216', '32', '19', '171']","[9671, 2751, 881, 8659]"
58972,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,...,101071,2151-03-05,2151-03-06 09:10:00,0,"30480 days, 0:00:00",83.506849,[9671],['Mech ventil '],['216'],[9671]
58973,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,RENAL CANCER/SDA,0,...,122631,2200-09-12,2200-09-20 12:08:00,8,"15534 days, 0:00:00",42.558904,"[4719, 5551, 4576, 4594, 1742]","['Appendectomy ', 'Nephrectomy ', 'Co...","['80', '104', '78', '96', '231']","[4719, 5551, 4576, 4594, -1]"
58974,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,S/P FALL,0,...,170407,2128-11-11,2128-12-22 13:11:00,41,"22195 days, 0:00:00",60.808219,"[9671, 3893, 9604, 966]","['Mech ventil ', 'Ot vasc cath ', 'Me...","['216', '54', '216', '223']","[9671, 3893, 9604, 966]"


In [37]:
print(grouped_adm.shape)
print(merge_proc.SUBJECT_ID.nunique())
merge_proc.HADM_ID.nunique()

(19993, 23)
7537


19993

In [38]:
merge_diag = pd.merge(merge_proc, grouped_diag, on=['SUBJECT_ID', 'HADM_ID'], how='left')
merge_diag.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,...,ICD9_CODE_procedures,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc,CCS CODES_diag,LEVE3 CODES,icd9_category,cat_threshold .75_diag,CCS CATEGORY DESCRIPTION_diag,ICD9_CODE_diagnosis
0,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,"[3613, 3615, 3961, 8872, 9904, 9905, 9907]","['CABG ', 'CABG ', 'Ex...","['44', '44', '50', '193', '222', '222', '222']","[3613, 3615, 3961, 8872, 9904, 9905, 9907]","[101, 101, 96, 101, 53, 98, 164, 94]","[414, 411, 424, V45, 272, 401, 600, 389]","[Diseases of the Circulatory System, Diseases ...","[41401, 4111, 4241, V4582, 2724, 4019, Otro, 3...","['Coron athero', 'Coron athero', 'Hrt valve dx...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ..."
1,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,...,[151],['Inc/exc CNS '],['1'],[0151],"[47, 95, 83, 96, 98, 53, 53, 101, 259, 663]","[225, 348, 780, 424, 401, 272, 272, V45, V45, ...","[Neoplasms, Diseases of the Nervous System and...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","['Ot bnign neo', 'Oth nerv dx', 'Epilepsy/cnv'...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4..."
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,...,"[3723, 8856, 8852]","['Cardiac cath ', 'Cardiac cath ', 'Ca...","['47', '47', '47']","[3723, 8856, 8852]","[100, 108, 97, 106, 238, 105, 101, 2616]","[410, 428, 425, 427, 997, 426, 414, E87]","[Diseases of the Circulatory System, Diseases ...","[41071, 4280, 4254, 42731, 9971, Otro, 41401, ...","['Acute MI', 'chf;nonhp', 'Carditis', 'Dysrhyt...","[41071, 4280, 4254, 42731, 9971, 4260, 41401, ..."
3,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRADYCARDIA,0,...,"[3772, 3783, 8945]","['Pacemaker ', 'Pacemaker ', 'Ot...","['48', '48', '227']","[3772, 3783, 8945]","[106, 108, 105, 101, 257, 108, 48, 88]","[427, 428, 426, 414, V58, 428, 244, 365]","[Diseases of the Circulatory System, Diseases ...","[42789, 42822, Otro, 41401, V5861, 4280, 2449,...","['Dysrhythmia', 'chf;nonhp', 'Conduction', 'Co...","[42789, 42822, 4263, 41401, V5861, 4280, 2449,..."
4,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,"[3615, 3961, 8856, 8853, 3722, 8872, 3614, 8845]","['CABG ', 'Extrcor circ ', 'Ca...","['44', '50', '47', '47', '47', '193', '44', '1...","[3615, 3961, 8856, 8853, 3722, 8872, 3614, 8845]","[101, 101, 127, 98, 663, 138, 164, 32, 162]","[414, 411, 496, 401, 305, 530, 600, V10, 596]","[Diseases of the Circulatory System, Diseases ...","[41401, 4111, 496, 4019, 3051, 53081, Otro, V1...","['Coron athero', 'Coron athero', 'COPD', 'HTN'...","[41401, 4111, 496, 4019, 3051, 53081, 60000, V..."


In [39]:
print(merge_diag.shape)
print(merge_diag.SUBJECT_ID.nunique())
merge_diag.HADM_ID.nunique()

(19993, 33)
7537


19993

In [40]:
merge_drug = pd.merge(merge_diag, grouped_drugs, on=['SUBJECT_ID', 'HADM_ID'], how='left')
merge_drug

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,...,cat_threshold .95 most frequent_proc,CCS CODES_diag,LEVE3 CODES,icd9_category,cat_threshold .75_diag,CCS CATEGORY DESCRIPTION_diag,ICD9_CODE_diagnosis,DRUG,ATC4,ATC3
0,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,"[3613, 3615, 3961, 8872, 9904, 9905, 9907]","[101, 101, 96, 101, 53, 98, 164, 94]","[414, 411, 424, V45, 272, 401, 600, 389]","[Diseases of the Circulatory System, Diseases ...","[41401, 4111, 4241, V4582, 2724, 4019, Otro, 3...","['Coron athero', 'Coron athero', 'Hrt valve dx...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[Potassium Chloride, Potassium Chloride, Docus...","[A12BA, A12BA, A06AA, A06AA, A02BA, A02BA, B05...","[A12B, A12B, A06A, A06A, A02B, A02B, B05C, C07..."
1,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,...,[0151],"[47, 95, 83, 96, 98, 53, 53, 101, 259, 663]","[225, 348, 780, 424, 401, 272, 272, V45, V45, ...","[Neoplasms, Diseases of the Nervous System and...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","['Ot bnign neo', 'Oth nerv dx', 'Epilepsy/cnv'...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[Heparin, 5% Dextrose, 0.9% Sodium Chloride, 0...","[B01AB, B05CX, A12CA, A12CA, A06AB, A02BC, N02...","[B01A, B05C, A12C, A12C, A06A, A02B, N02B, C02..."
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,...,"[3723, 8856, 8852]","[100, 108, 97, 106, 238, 105, 101, 2616]","[410, 428, 425, 427, 997, 426, 414, E87]","[Diseases of the Circulatory System, Diseases ...","[41071, 4280, 4254, 42731, 9971, Otro, 41401, ...","['Acute MI', 'chf;nonhp', 'Carditis', 'Dysrhyt...","[41071, 4280, 4254, 42731, 9971, 4260, 41401, ...","[Metoprolol, Furosemide, Furosemide, Heparin S...","[C07AB, C03CA, C03CA, B01AB, N02BE, C02DB, A01...","[C07A, C03C, C03C, B01A, N02B, C02D, A01A, C07..."
3,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRADYCARDIA,0,...,"[3772, 3783, 8945]","[106, 108, 105, 101, 257, 108, 48, 88]","[427, 428, 426, 414, V58, 428, 244, 365]","[Diseases of the Circulatory System, Diseases ...","[42789, 42822, Otro, 41401, V5861, 4280, 2449,...","['Dysrhythmia', 'chf;nonhp', 'Conduction', 'Co...","[42789, 42822, 4263, 41401, V5861, 4280, 2449,...","[Warfarin, Acetaminophen, Ondansetron, Midazol...","[B01AA, N02BE, A04AA, N05CD, A01AD, A01AD, C03...","[B01A, N02B, A04A, N05C, A01A, A01A, C03C, A01..."
4,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,...,"[3615, 3961, 8856, 8853, 3722, 8872, 3614, 8845]","[101, 101, 127, 98, 663, 138, 164, 32, 162]","[414, 411, 496, 401, 305, 530, 600, V10, 596]","[Diseases of the Circulatory System, Diseases ...","[41401, 4111, 496, 4019, 3051, 53081, Otro, V1...","['Coron athero', 'Coron athero', 'COPD', 'HTN'...","[41401, 4111, 496, 4019, 3051, 53081, 60000, V...","[Potassium Chloride, Heparin, Docusate Sodium,...","[A12BA, B01AB, A06AA, A06AA, A06AA, A06AB, A02...","[A12B, B01A, A06A, A06A, A06A, A06A, A02B, A02..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19988,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,GASTROPARESIS\PLACEMENT OF G-TUBE **REMOTE WES...,0,...,[4311],"[141, 155]","[536, 564]","[Diseases of the Digestive System, Diseases of...","[5363, Otro]","['Ot dx stomch', 'Other GI dx']","[5363, 5641]",NaN,NaN,NaN
19989,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,SHORT GUT SYNDROME/SDA,0,...,[9702],"[238, 238, 141, 148, 251, 2616, 141, 210, 138,...","[536, 579, 537, 567, 789, E87, 536, 710, 530, ...","[Diseases of the Digestive System, Diseases of...",

In [41]:
print(merge_drug.shape)
print(merge_drug.SUBJECT_ID.nunique())
merge_drug.HADM_ID.nunique()

(19993, 36)
7537


19993

In [42]:
pd.set_option('display.max_columns', None)

merge_drug

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,EXPIRE_FLAG,LOS,LOS_right,DOB,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,LOSR,LOSRD,age,year_age,ICD9_CODE_procedures,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc,CCS CODES_diag,LEVE3 CODES,icd9_category,cat_threshold .75_diag,CCS CATEGORY DESCRIPTION_diag,ICD9_CODE_diagnosis,DRUG,ATC4,ATC3
0,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,0,"[9.29, 30.34, 99.15, None]",[1.2641],2082-07-17,23,152223,2153-09-03,2153-09-08 19:10:00,5 days 11:55:00,5,"25980 days, 0:00:00",71.178082,"[3613, 3615, 3961, 8872, 9904, 9905, 9907]","['CABG ', 'CABG ', 'Ex...","['44', '44', '50', '193', '222', '222', '222']","[3613, 3615, 3961, 8872, 9904, 9905, 9907]","[101, 101, 96, 101, 53, 98, 164, 94]","[414, 411, 424, V45, 272, 401, 600, 389]","[Diseases of the Circulatory System, Diseases ...","[41401, 4111, 4241, V4582, 2724, 4019, Otro, 3...","['Coron athero', 'Coron athero', 'Hrt valve dx...","[41401, 4111, 4241, V4582, 2724, 4019, 60000, ...","[Potassium Chloride, Potassium Chloride, Docus...","[A12BA, A12BA, A06AA, A06AA, A02BA, A02BA, B05...","[A12B, A12B, A06A, A06A, A02B, A02B, B05C, C07..."
1,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,0,M,0,"[64.09, 28.47, 5.95, 63.97, None]",[1.1862],2082-07-17,23,124321,2157-10-18,2157-10-25 14:00:00,6 days 18:26:00,6,"27486 days, 0:00:00",75.304110,[151],['Inc/exc CNS '],['1'],[0151],"[47, 95, 83, 96, 98, 53, 53, 101, 259, 663]","[225, 348, 780, 424, 401, 272, 272, V45, V45, ...","[Neoplasms, Diseases of the Nervous System and...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","['Ot bnign neo', 'Oth nerv dx', 'Epilepsy/cnv'...","[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...","[Heparin, 5% Dextrose, 0.9% Sodium Chloride, 0...","[B01AB, B05CX, A12CA, A12CA, A06AB, A02BC, N02...","[B01A, B05C, A12C, A12C, A06A, A02B, N02B, C02..."
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,CHEST PAIN\CATH,0,M,1,"[1.4, 17.28, 28.67, None]",[.7198],1886-07-18,34,115799,2186-07-18,2186-07-20 16:00:00,1 days 23:14:00,1,"109573 days, 0:00:00",90.000000,"[3723, 8856, 8852]","['Cardiac cath ', 'Cardiac cath ', 'Ca...","['47', '47', '47']","[3723, 8856, 8852]","[100, 108, 97, 106, 238, 105, 101, 2616]","[410, 428, 425, 427, 997, 426, 414, E87]","[Diseases of the Circulatory System, Diseases ...","[41071, 4280, 4254, 42731, 9971, Otro, 41401, ...","['Acute MI', 'chf;nonhp', 'Carditis', 'Dysrhyt...","[41071, 4280, 4254, 42731, 9971, 4260, 41401, ...","[Metoprolol, Furosemide, Furosemide, Heparin S...","[C07AB, C03CA, C03CA, B01AB, N02BE, C02DB, A01...","[C07A, C03C, C03C, B01A, N02B, C02D, A01A, C07..."
3,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRADYCARDIA,0,M,1,"[.13, .59, 37.25, 24.95, None]",[1.5824],1886-07-18,34,144319,2191-02-23,2191-02-25 20:20:00,2 days 14:57:00,2,"111254 days, 0:00:00",90.000000,"[3772, 3783, 8945]","['Pacemaker ', 'Pacemaker ', 'Ot...","['48', '48', '227']","[3772, 3783, 8945]","[106, 108, 105, 101, 257, 108, 48, 88]","[427, 428, 426, 414, V58, 428, 244, 365]","[Diseases of the Circulatory System, Diseases ...","[42789, 42822, Otro, 41401, V5861, 4280, 2449,...","['Dysrhythmia', 'chf;nonhp', 'Conduction', 'Co...","[42789, 42822, 4263, 41401, V5861, 4280, 2449,...","[Warfarin, Acetaminophen, Ondansetron, Midazol...","[B01AA, N02BE, A04AA, N05CD, A01AD, A01AD, C03...","[B01A, N02B, A04A, N05C, A01A, A01A, C03C, A01..."
4,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,M,0,"[86.74, 2.82, 26.63, 72.96, None]",[1.1096],2061-08-17,36,182104,2131-04-30,2131-05-08 14:00:00,8 days 06:45:00,8,"2545

In [43]:
merge_drug.to_csv("data_preprocess.csv")

In [38]:
# este es el preprocess que solo considera  procedures
merge_proc.to_csv("data_preprocess_non_filtered.csv")

# Analisis of icd_9 == 9

In [152]:
The code `s_ids = pd.read_csv("ids_9.csv")["SUBJECT_ID"]` is reading a CSV file named "ids_9.csv" and extracting the values from the "SUBJECT_ID" column. It assigns these values to the variable `s_ids`.
s_ids = pd.read_csv("ids_9.csv")["SUBJECT_ID"]
s_hadmids = pd.read_csv("ids_9.csv")["HADM_ID"]

In [156]:
s_ids

0        6398
1       22432
2       22446
3        7676
4       16881
        ...  
3310     5362
3311    14975
3312     8463
3313     5362
3314     8463
Name: SUBJECT_ID, Length: 3315, dtype: int64

In [154]:

desired_ids = s_ids
condition = merge_drug['SUBJECT_ID'].isin(desired_ids)

# Apply the condition to filter the DataFrame
filtered_ds = merge_drug[condition]


In [155]:
filtered_ds

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,HOSPITAL_EXPIRE_FLAG,ICD9_CODE,CCS CATEGORY DESCRIPTION_proc,CCS CODES_proc,cat_threshold .95 most frequent_proc,CCS CODES_diag,LEVE3 CODES,icd9_category,cat_threshold .75_diag,CCS CATEGORY DESCRIPTION_diag,DRUG,ATC4,ATC3
4,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,0,"[3615, 3961, 8856, 8853, 3722, 8872, 3614, 8845]","['CABG ', 'Extrcor circ ', 'Ca...","['44', '50', '47', '47', '47', '193', '44', '1...","[3615, 3961, 8856, 8853, 3722, 8872, 3614, 8845]","[101, 101, 127, 98, 663, 138, 164, 32, 162]","[414, 411, 496, 401, 305, 530, 600, V10, 596]","[Diseases of the Circulatory System, Diseases ...","[41401, 4111, 496, 4019, 3051, 53081, Otro, V1...","['Coron athero', 'Coron athero', 'COPD', 'HTN'...","[Potassium Chloride, Heparin, Docusate Sodium,...","[A12BA, B01AB, A06AA, A06AA, A06AA, A06AB, A02...","[A12B, B01A, A06A, A06A, A06A, A06A, A02B, A02..."
5,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,0,"[3893, 3479, 7761, 7761, 8674, 3403, 3932, 990...","['Ot vasc cath ', 'OR respirato ', 'Ex...","['54', '42', '142', '142', '172', '42', '61', ...","[3893, 3479, 7761, 7761, 8674, 3403, 3932, 990...","[238, 238, 238, 118, 237, 127, 101, 101, 98, 1...","[998, 998, 415, 453, 996, 496, 414, V45, 401, ...","[Injury and Poisoning, Injury and Poisoning, D...","[99831, 99811, Otro, Otro, 99672, 496, 41401, ...","['Complic proc', 'Complic proc', 'Complic proc...","[Potassium Chloride, Potassium Chloride, Potas...","[A12BA, A12BA, A12BA, A12BA, B01AB, B01AB, A06...","[A12B, A12B, A12B, A12B, B01A, B01A, A06A, A06..."
6,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,0,"[9671, 9604, 3479, 544, 5361, 7861, 9977]","['Mech ventil ', 'Mech ventil ', 'OR...","['216', '216', '42', '99', '86', '148', '231']","[9671, 9604, 3479, 544, 5361, Otro, 9977]","[143, 238, 131, 122, 238, 130, 101, 127, 98, 1...","[553, 415, 518, 486, 997, 518, 414, 496, 401, ...","[Diseases of the Digestive System, Diseases of...","[Otro, Otro, 5185, 486, 99739, 5180, 41400, 49...","['Abdom hernia', 'Complic proc', 'Adlt resp fl...","[Heparin, Heparin, Heparin, Heparin, Heparin, ...","[B01AB, B01AB, B01AB, B01AB, B01AB, C03CA, C03...","[B01A, B01A, B01A, B01A, B01A, C03C, C03C, C03..."
16,368,105889,2137-07-11 17:56:00,2137-07-16 16:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,NOT SPECIFIED,WIDOWED,0,NaN,NaN,NaN,NaN,"[122, 127, 100, 157, 158, 117, 59, 138, 651]","[486, 491, 410, 584, 585, 458, 280, 530, 300]","[Diseases of the Respiratory System, Diseases ...","[486, 49121, 41071, 5849, Otro, 4589, 2809, 53...","['Pneumonia', 'COPD', 'Acute MI', 'Ac renl fai...","[Prednisone, Prednisone, Acetaminophen, Hepari...","[A07EA, A07EA, N02BE, B01AB, B01AB, B01AB, A06...","[A07E, A07E, N02B, B01A, B01A, B01A, A06A, A12..."
17,368,138061,2139-12-16 19:48:00,2139-12-22 16:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,NOT SPECIFIED,WIDOWED,0,[9904],['Blood transf '],['222'],[9904],"[100, 122, 131, 108, 55, 99, 55, 127, 101, 100...","[410, 486, 518, 428, 276, 403, 276, 491, 414, ...","[Diseases of the Circulatory System, Diseases ...","[41071, 486, 51882, 4280, 2762, 40391, Otro, O...","['Acute MI', 'Pneumonia', 'Adlt resp fl', 'chf...","[Potassium Chloride, Prednisone, Prednisone, P...","[A12BA, A07EA, A07EA, A07EA, B01AB, A06AA, A06...","[A12B, A07E, A07E, A07E, B01A, A06A, A06A, A12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19983,98744,184410,2127-06-30 20:19:00,2127-07-04 14:35:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,ENGL,NOT SPE